In [1]:
import numpy as np

In [4]:
y = np.array([1, 0, 0])
z = np.array([0.0, 0.1, -0.1])
y_pred = np.exp(z)/np.exp(z).sum()
loss = (-y * np.log(y_pred)).sum()
print(loss)

1.101942848229244


In [7]:
import torch

criterion = torch.nn.CrossEntropyLoss()



Y = torch.LongTensor([2, 0, 1])
Y_pred1 = torch.Tensor([[0.1, 0.2, 0.9],
                        [1.1, 0.1, 0.2],
                        [0.2, 2.1, 0.1]])

Y_pred2 = torch.Tensor([[0.8, 0.2, 0.3],
                        [0.2, 0.3, 0.5],
                        [0.2, 0.2, 0.5]])


l1 = criterion(Y_pred1, Y)
l2 = criterion(Y_pred2, Y)
print('Batch Loss1 = ',l1.data, "\nBatch Loss2 = ", l2.data)

Batch Loss1 =  tensor(0.4966) 
Batch Loss2 =  tensor(1.2389)


crossEntryLoss vs. NLLLoss  
交叉熵损失和负log损失区别 

In [1]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F 
import torch.optim as optim

In [2]:
batch_size = 64
transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307, ), (0.3081,)) # MNIST mean and std
])

In [3]:
train_dataset = datasets.MNIST(root='./data/mnist',
                                train=True,
                                transform=transforms,
                                download=True)

test_dataset = datasets.MNIST(root='./data/mnist',
                                train=False,
                                transform=transform,
                                download=True)

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset,
                          batch_size=batch_size,
                          shuffle=False)


 69%|██████▉   | 6880256/9912422 [11:51<05:13, 9665.14it/s]  


RuntimeError: File not found or corrupted.

In [ ]:
class Net(torch.nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.l1 = torch.nn.Linear(784, 512)
    self.l2 = torch.nn.Linear(512, 256)
    self.l3 = torch.nn.Linear(256, 128)
    self.l4 = torch.nn.Linear(128, 64)
    self.l5 = torch.nn.Linear(64, 10)

  
  def forward(self, x):
    x = x.view(-1, 784)
    x = F.relu(self.l1(x))
    x = F.relu(self.l2(x))
    x = F.relu(self.l3(x))
    x = F.relu(self.l4(x))
    return self.l5(x)

model =Net()



In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


In [ ]:
def train(epoch):
  running_loss = 0.0
  for batch_idx, data in enumerate(train_loader, 0):
    inputs, target = data
    optimizer.zero_grad()

    outputs = model(inputs)
    loss = criterion(outputs, target)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if batch_idx % 300 == 299:

      print("[%d, %5d] loss: %.3f "%(epoch + 1, batch_idx + 1, running_loss / 300))
      running_loss = 0.0

In [ ]:
def test():
  correct = 0
  total = 0
  with torch.no_grad(): # 表示不再计算梯度
    for data in test_loader:
      images, labels = data
      outputs = model(images)
      _, predicted = torch.max(outputs.data, dim=1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
  
  print("Accuracy on test set: %d %%"%(100 * correct / total))

In [ ]:
if __name__ == '__main__':
  for epoch in range(10):
    train(epoch)
    test()